In [1]:
import numpy as np
import argparse
import cv2

from tqdm import tqdm
import sys
sys.path.append("/Users/rcarlson/Documents/GitHub/lasagna3/snakes")
import mahotas
from ops.imports_ipython import *
from scipy.spatial.distance import cdist


import skimage
import tensorflow as tf

/home/rcarlson/OpticalPooledScreens/ops/annotate.py:20: UserWarning: visitor font not found at /home/rcarlson/OpticalPooledScreens/ops/visitor1.ttf
  warnings.warn('visitor font not found at {0}'.format(VISITOR_PATH))


In [11]:
df = pd.read_hdf('/home/rcarlson/mountb/mergehdfs2/dfall_key_nona.hdf')

In [12]:
df.loc[:,df.columns.str.startswith("dapi")]

dapi_corrch1_nuclear_corr  dapi_corrch2_nuclear_corr  \
1                          0.825456                   0.941051   
2                          0.383439                   0.514844   
3                          0.781937                   0.915194   
4                         -0.608751                   0.876683   
5                         -0.251165                   0.008873   
...                             ...                        ...   
10605514                   0.556273                   0.712087   
10605515                  -0.553204                  -0.173183   
10605516                  -0.055369                   0.293678   
10605517                   0.411618                   0.791005   
10605518                  -0.236385                   0.426923   

          dapi_corrch3_nuclear_corr  dapi_corrch4_nuclear_corr  \
1                          0.888915                  -0.291652   
2                          0.311178                   0.023699   
3                          0.746572                  -0.411696   
4                          0.577062                  -0.755424   
5                         -0.364610                  -0.313472   
...                             ...                        ...   
10605514                   0.536526                  -0.460566   
10605515                  -0.542576                  -0.345997   
10605516                   0.061933                   0.006356   
10605517                   0.580522                  -0.327935   
10605518                  -0.138199                  -0.492976   

          dapi_corrch5_nuclear_corr  dapi_corrch6_nuclear_corr  \
1                         -0.202854                  -0.179432   
2                          0.173520                   0.034548   
3                         -0.090782                   0.448206   
4                         -0.776978                  -0.450777   
5                         -0.303239                  -0.132522   
...                             ...                        ...   
10605514                  -0.468065                   0.183910   
10605515                  -0.429596                  -0.241242   
10605516                  -0.273385                   0.189908   
10605517                  -0.597198                   0.260713   
10605518                  -0.493697                   0.119581   

          dapi_nuclear_mean  dapi_nuclear_median  dapi_nuclear_int  \
1               9593.665039              10104.0           4614553   
2               3219.816406               3279.0           1577710   
3              10268.973633              10560.0           3891941   
4               9521.233398               9717.0           4693968   
5               9399.060547               9792.0           4050995   
...                     ...                  ...               ...   
10605514        5432.056641               5400.0           2292328   
10605515        7437.725586               7344.0           2357759   
10605516        5019.064941               5033.0           3869699   
10605517        5405.458008               5411.5           2432456   
10605518        6957.755859               7012.0           4077245   

          dapi_nuclear_min  ...  dapi_pftas_cyto45  dapi_pftas_cyto46  \
1                     4448  ...           0.000000           0.668067   
2                     1310  ...           0.000000           0.646789   
3                     4884  ...           0.000000           0.571429   
4                     4972  ...           0.000000           0.475285   
5                     4477  ...           0.000000           0.484848   
...                    ...  ...                ...                ...   
10605514              2828  ...           0.000000           0.579167   
10605515              3676  ...           0.001938           0.631769   
10605516              2560  ...           0.002611           0.678733   
10605517              2820  ...           0.000000          

In [46]:
maxes = list(df.loc[:,df.columns.str.endswith("cell_max")].quantile(.995))

In [47]:
(maxes)

[16945.0, 36575.984999999404, 23873.0, 14054.0, 62946.0, 35087.0, 50155.0]

In [ ]:
df = pd.read_hdf('/home/rcarlson/mountb/mergehdfs2/dfnew_dropped_corrssover08')


def f(x):
    d = {}
    d['min_dist'] = np.partition(cdist(x[['i','j']].values, x[['i','j']].values),1,axis=1)[:,1]
    d['cell'] = x['cell']
    d['plate'] = x['plate']
    d['gene_symbol'] = x['gene_symbol']
    d['cell_barcode_0'] = x['cell_barcode_0']
    d['i'] = x['i']
    d['j'] = x['j']

    return pd.DataFrame([d['min_dist'],d['cell'],d['gene_symbol'],d['cell_barcode_0'],d['i'],d['j']]).transpose()#pd.Series(d, index=['min_dist','cell'])





In [30]:
tmp = df.iloc[:,:10].copy()
to_keep = tmp.groupby(['plate','well','site']).cell.count().loc[lambda x: x>1].reset_index()
tmp = pd.merge(tmp, to_keep, how = 'right', on = ['plate','well','site'])

In [36]:
tmp.shape[0]/df.shape[0] # kept vast majority of cells

0.9999326670132997

In [40]:
tmp['cell'] = tmp['cell_x']
tmp = (tmp
       .groupby(['plate','well','site'])
       .apply(f).reset_index())


 
tmp.drop('level_3',inplace = True, axis=1)
tmp.columns = ['plate','well','Site','min_dist','cell','gene','cell_barcode_0','i','j']





In [55]:
tmp[tmp.min_dist > 13].shape[0]/tmp.shape[0] ## keep ~88% of cells, aim to keep single-cell crop 26x26 ish



0.8881839244307085

In [ ]:

df_26 = tmp[tmp.min_dist > 13]

grped = df_26.groupby(['cell_barcode_0']).count()
few_cells = list(grped[grped.cell < 25].index)
print(len(few_cells))
df_26 = df_26[~df_26.cell_barcode_0.isin(few_cells)]

df_26.to_hdf('/home/rcarlson/mountb/m40-m42/df_26.hdf',key='x')





In [3]:
df_26 = pd.read_hdf('/home/rcarlson/mountb/m40-m42/df_26.hdf')

In [3]:
og_ij = pd.read_hdf('/home/rcarlson/mountb/mergehdfs2/dfall.hdf')



In [4]:
og_ij[['plate', 'well', 'site','cell', 'i', 'j']].to_hdf('/home/rcarlson/mountb/mergehdfs2/dfall_ijonly.hdf', key = 'x')



In [5]:
df_26['site'] = df_26.Site
og_ij = pd.read_hdf('/home/rcarlson/mountb/mergehdfs2/dfall_ijonly.hdf')

In [6]:
#, start = 0, stop = 5,  columns =  ['plate', 'well', 'site', 'i', 'j'])
df_26 = pd.merge(df_26, og_ij, on = ['plate', 'well', 'site', 'cell'], how = 'left')


In [11]:
df_26['i'] = df_26.i_y
df_26['j'] = df_26.j_y
df_26.to_hdf('/home/rcarlson/mountb/m40-m42/df_26.hdf',key='x')



/home/rcarlson/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:2620: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->Index(['plate', 'well', 'min_dist', 'cell', 'gene', 'cell_barcode_0', 'i_x',
       'j_x'],
      dtype='object')]

  encoding=encoding,


In [2]:
df_26 = pd.read_hdf('/home/rcarlson/mountb/m40-m42/df_26.hdf')

In [9]:
from natsort import natsorted 
def save_singlecells_crop(plate, well, nchannels, df_cells_interest, dist_from_center = 23):
    border_ct = 0
      # loc only for purposes of defining sites/wells of interest
    # df should contain all cells of interest with i/j indices
    df_cells_interest = df_cells_interest.query('plate == @plate & well == @well')
    files = natsorted(glob('/home/rcarlson/mountb/%s/Well%s*.aligned.tif'%(plate, well)))
    site_list = []
    wells = []
    sites = []
    for n in range(len(files)):
        well = re.findall("/(.*)_site(.*)", (files)[n])[0][0].split('Well')[-1]
        site = re.findall("/(.*)_site(.*)", (files)[n])[0][1].split('.')[0]
        wells.append(well)
        sites.append(site)
        site_list.append([str(well) + '_site' + str(site)+'.'])
    
    cellct = 0
    mastercellct = 0

    
    print('# sites')
    print(len(site_list))
    for z in range(len(site_list)):
        print(mastercellct)
        print(site_list[z])
        well = wells[z]
        site = int(sites[z])
        file =  read('/home/rcarlson/mountb/%s/Well%s_site%s.aligned.tif' %(plate, well, site))
        tmp = df_cells_interest[(df_cells_interest.well == well) & (df_cells_interest.Site == site)]
        ils = list(tmp.i.apply(int))
        jls = list(tmp.j.apply(int))
        cells = list(tmp.cell)
        for i in range((tmp.shape[0])):
            
            
            bounds_cell = np.array((ils[i]-dist_from_center,ils[i]+dist_from_center,
                                    jls[i]-dist_from_center,jls[i]+dist_from_center))
            if (sum(bounds_cell < 0) != 0) | (sum(bounds_cell > 1480) != 0):
                continue # ignore cells on border
                border_ct += 1
            
            cellct += 1
            mastercellct += 1

            tosave = file[:,bounds_cell[0]:bounds_cell[1],bounds_cell[2]:bounds_cell[3]][0:nchannels].astype('float32')
            
            for ch in range(nchannels):
                image_resized = skimage.transform.resize(tosave[ch], (64, 64),
                               anti_aliasing=True, mode = 'constant').astype('float32')
                f = name({}, ext = "ch%s.tif"%ch, mag = "%s_%s"%(plate, cells[i]), well = well, site = site, 
                subdir = "/home/rcarlson/mountb/m40-m42/process/og_single_crops")
                print(np.array([image_resized]).shape)
                save(f, tosave[ch])


                
        read.reset
        
        



In [10]:
plate = "M40A"
for well in ["A1"]:

    save_singlecells_crop(
                 plate = plate,
                 nchannels = 7, well = well, 
                df_cells_interest = df_26.query('well == @well & site < 30'))

# sites
1281
0
['A1_site0.']
53
['A1_site1.']
252
['A1_site2.']
318
['A1_site3.']
479
['A1_site4.']
511
['A1_site5.']
664
['A1_site6.']
687
['A1_site7.']
740
['A1_site8.']
740
['A1_site9.']
757
['A1_site10.']
757
['A1_site11.']
826
['A1_site12.']
829
['A1_site13.']
887
['A1_site14.']
887
['A1_site15.']
944
['A1_site16.']
969
['A1_site17.']
1094
['A1_site18.']
1094
['A1_site19.']
1094
['A1_site20.']
1217
['A1_site21.']
1248
['A1_site22.']
1406
['A1_site23.']
1417
['A1_site24.']
1579
['A1_site25.']
1600
['A1_site26.']
1748
['A1_site27.']
1748
['A1_site28.']
1893
['A1_site29.']
1925
['A1_site30.']
1925
['A1_site31.']
1925
['A1_site32.']
1925
['A1_site33.']
1925
['A1_site34.']
1925
['A1_site35.']
1925
['A1_site36.']
1925
['A1_site37.']
1925
['A1_site38.']
1925
['A1_site39.']
1925
['A1_site40.']
1925
['A1_site41.']
1925
['A1_site42.']
1925
['A1_site43.']
1925
['A1_site44.']
1925
['A1_site45.']
1925
['A1_site46.']
1925
['A1_site47.']
1925
['A1_site48.']
1925
['A1_site49.']
1925
['A1_site50.'

KeyboardInterrupt: 

## Run pretrained network

In [7]:
from keras.applications import imagenet_utils
from keras.applications import Xception # TensorFlow ONLY

from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

from keras.models import Model

# Source model
model = Xception(weights = 'imagenet')

# remove last layer
bottleneck_model = Model(inputs=model.input, outputs=model.get_layer('avg_pool').output)
print(bottleneck_model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [5]:

import gc

def preprocess_img(imgname):
        print(imgname)
        imgs = (read(imgname))
        print(imgs.shape)
        mean = imgs.mean(axis = (1,2))
        sd = imgs.std(axis = (1,2))
        mins = imgs.min(axis = (1,2))
        maxs = imgs.max(axis = (1,2))
        
        mean = np.mean(mean)
        sd = np.mean(sd)
        
        print('mean, sd')
        print(mean, sd)
        
        imgs = (imgs - mean)/sd

        imlist = []
        for i in range(imgs.shape[0]):
            if i % 500 == 0:
                print(i)
            img = imgs[i] - mins[i]
            img = img * 255/maxs[i]
            img = np.expand_dims(img, axis = -1)
            img = np.repeat(img, 3, axis = -1)
            img = keras.applications.xception.preprocess_input(img)
            img = np.expand_dims(img,axis = 0)
            imlist.append(img)
        
        print('preprocessing done')

        return imlist
    
def predict_features(imgs):
    imgs = tf.data.Dataset.from_tensor_slices(imgs)
    print('tf dataset created')
    features = bottleneck_model.predict(imgs, use_multiprocessing = True, batch_size = 100, verbose = 1, workers = 2)
    return features


from tqdm.contrib.concurrent import process_map
import keras
def batch_preprocess_predict(file, start, end, plate, well):
    imgs = preprocess_img(file)
    print(len(imgs))

    feats = predict_features(imgs)
    (pd.DataFrame(feats)
        .to_pickle('/home/rcarlson/mountb/m40-m42/process/xception/feats_'+plate+'_'+well+'_'+str(start)+'_'+str(end)+'.ch'+str(channel)+'.pkl'))

In [3]:
from natsort import natsorted 
def save_singlecells_finish(plate, well, start, nchannels, df_cells_interest, dist_from_center = 23):
    border_ct = 0
      # loc only for purposes of defining sites/wells of interest
    # df should contain all cells of interest with i/j indices
    df_cells_interest = df_cells_interest.query('plate == @plate & well == @well')
    files = natsorted(glob('/home/rcarlson/mountb/%s/Well%s*.aligned.tif'%(plate, well)))
    site_list = []
    wells = []
    sites = []
    for n in range(len(files)):
        well = re.findall("/(.*)_site(.*)", (files)[n])[0][0].split('Well')[-1]
        site = re.findall("/(.*)_site(.*)", (files)[n])[0][1].split('.')[0]
        wells.append(well)
        sites.append(site)
        site_list.append([str(well) + '_site' + str(site)+'.'])
    
    cellct = 0
    mastercellct = start
    cells2k = []
    savels = []
    
    print('# sites')
    print(len(site_list))
    for z in range(len(site_list)):
        print(mastercellct)
        print(site_list[z])
        well = wells[z]
        site = int(sites[z])
        if site not in pd.unique(df_cells_interest.site):
            continue
        file =  read('/home/rcarlson/mountb/%s/Well%s_site%s.aligned.tif' %(plate, well, site))
        tmp = df_cells_interest[(df_cells_interest.well == well) & (df_cells_interest.Site == site)]
        ils = list(tmp.i.apply(int))
        jls = list(tmp.j.apply(int))
        cells = list(tmp.cell)
        for i in range((tmp.shape[0])):
            bounds_cell = np.array((ils[i]-dist_from_center,ils[i]+dist_from_center,
                                    jls[i]-dist_from_center,jls[i]+dist_from_center))
            if (sum(bounds_cell < 0) != 0) | (sum(bounds_cell > 1480) != 0):
                continue # ignore cells on border
                border_ct += 1
            
            cellct += 1
            mastercellct += 1

            tosave = file[:,bounds_cell[0]:bounds_cell[1],bounds_cell[2]:bounds_cell[3]][0:nchannels].astype('float32')
            
            ### RESIZE ###
            
            image_resized0 = skimage.transform.resize(tosave[0], (299 , 299),
                               anti_aliasing=True, mode = 'constant').astype('float32')
            image_resized1 = skimage.transform.resize(tosave[1], (299, 299),
                                 anti_aliasing=True, mode = 'constant').astype('float32')
            image_resized2 = skimage.transform.resize(tosave[2], (299, 299),
                               anti_aliasing=True, mode = 'constant').astype('float32')
            image_resized3 = skimage.transform.resize(tosave[3], (299, 299),
                               anti_aliasing=True, mode = 'constant').astype('float32')           
            image_resized4 = skimage.transform.resize(tosave[4], (299, 299),
                               anti_aliasing=True, mode = 'constant').astype('float32')
            image_resized5 = skimage.transform.resize(tosave[5], (299, 299),
                               anti_aliasing=True, mode = 'constant').astype('float32')
            image_resized6 = skimage.transform.resize(tosave[6], (299, 299),
                               anti_aliasing=True, mode = 'constant').astype('float32')
            
            tosave = np.array((image_resized0,image_resized1,image_resized2,image_resized3,image_resized4,
                              image_resized5,image_resized6))
            cells2k.append(tosave)
            
            savename = '/home/rcarlson/mountb/%s/Well%s_site%s.aligned.tif' %(plate, well, site) + "_%s.cell" %(cells[i])
            savels.append(savename)

            if cellct == 2000:
                cells2k = np.array(cells2k)
                print('saving')
                
                f = name({}, ext = "ch", mag = "%s_%s_%s"%(plate, mastercellct-cellct,
                                                                                    mastercellct), well = well, site = site, 
                     subdir = "/home/rcarlson/mountb/m40-m42/process/single_cells")
                
                
                for ch in range(cells2k.shape[1]):
                    print(f + str(ch) + '.tif')
                    save(f + str(ch) + '.tif', cells2k[:,ch,:,:])
                  
                pd.Series(savels).to_csv("/home/rcarlson/mountb/m40-m42/process/cells_ls/%s_%s_%s_%s.csv"%(plate, well, mastercellct-cellct,mastercellct))
                savels = []
                cells2k = []
                cellct = 0

            print(tmp.shape)
            if (z == len(site_list)-1) & (i == tmp.shape[0]-1):
                cells2k = np.array(cells2k)
                print('saving last site and cell')
                f = name({}, ext = "ch", mag = "%s_%s_%s"%(plate, mastercellct-cellct,
                                                                                    mastercellct), well = well, site = site, 
                     subdir = "/home/rcarlson/mountb/m40-m42/process/single_cells")
                
                for ch in range(cells2k.shape[1]):
                    save(f + str(ch) + '.tif', cells2k[:,ch,:,:])
                
                pd.Series(savels).to_csv("/home/rcarlson/mountb/m40-m42/process/cells_ls/%s_%s_%s_%s.csv"%(plate, well, mastercellct-cellct,mastercellct))
                
                print(np.array(cells2k).shape)

                
        read.reset
        
        



In [27]:
ix = df_26.query('well == @well & plate == @plate & site == @site & cell == @cell').index[0]
ofint = df_26.copy().query('well == @well & plate == @plate').iloc[ix+1:,:]
ofint.head()

Empty DataFrame
Columns: [plate, well, Site, min_dist, cell, gene, cell_barcode_0, i_x, j_x, site, i_y, j_y, i, j]
Index: []

In [4]:
plate = "M40A"
for well in ["B1"]:#,"A3","B1","B2","B3"]:

    save_singlecells(
                 plate = plate,
                 nchannels = 7, well = well, 
                df_cells_interest = df_26.query('well == @well'))

# sites
1281
0
['B1_site0.']
44
['B1_site1.']
145
['B1_site2.']
173
['B1_site3.']
304
['B1_site4.']
364
['B1_site5.']
377
['B1_site6.']
401
['B1_site7.']
504
['B1_site8.']
538
['B1_site9.']
724
['B1_site10.']
754
['B1_site11.']
821
['B1_site12.']
821
['B1_site13.']
911
['B1_site14.']
911
['B1_site15.']
1037
['B1_site16.']
1053
['B1_site17.']
1093
['B1_site18.']
1093
['B1_site19.']
1182
['B1_site20.']
1547
['B1_site21.']
1766
['B1_site22.']
saving
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_0_2000_B1_Site-22.ch0.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_0_2000_B1_Site-22.ch1.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_0_2000_B1_Site-22.ch2.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_0_2000_B1_Site-22.ch3.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_0_2000_B1_Site-22.ch4.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_0_2000_B1_Site-22.ch5.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_0

/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_18000_20000_B1_Site-121.ch3.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_18000_20000_B1_Site-121.ch4.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_18000_20000_B1_Site-121.ch5.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_18000_20000_B1_Site-121.ch6.tif
20035
['B1_site122.']
20375
['B1_site123.']
20565
['B1_site124.']
20930
['B1_site125.']
21137
['B1_site126.']
21526
['B1_site127.']
21793
['B1_site128.']
saving
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_20000_22000_B1_Site-128.ch0.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_20000_22000_B1_Site-128.ch1.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_20000_22000_B1_Site-128.ch2.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_20000_22000_B1_Site-128.ch3.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_20000_22000_B1_Site-128.ch4.tif
/home/rcarlson/mountb/m40-m42/process/single_ce

38496
['B1_site224.']
38622
['B1_site225.']
38838
['B1_site226.']
38962
['B1_site227.']
39204
['B1_site228.']
39273
['B1_site229.']
39479
['B1_site230.']
39525
['B1_site231.']
39680
['B1_site232.']
39710
['B1_site233.']
39883
['B1_site234.']
39883
['B1_site235.']
saving
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_38000_40000_B1_Site-235.ch0.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_38000_40000_B1_Site-235.ch1.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_38000_40000_B1_Site-235.ch2.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_38000_40000_B1_Site-235.ch3.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_38000_40000_B1_Site-235.ch4.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_38000_40000_B1_Site-235.ch5.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_38000_40000_B1_Site-235.ch6.tif
40030
['B1_site236.']
40030
['B1_site237.']
40337
['B1_site238.']
40445
['B1_site239.']
40671
['B1_site240.']
407

57064
['B1_site342.']
57325
['B1_site343.']
57497
['B1_site344.']
57545
['B1_site345.']
57773
['B1_site346.']
57845
['B1_site347.']
saving
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_56000_58000_B1_Site-347.ch0.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_56000_58000_B1_Site-347.ch1.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_56000_58000_B1_Site-347.ch2.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_56000_58000_B1_Site-347.ch3.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_56000_58000_B1_Site-347.ch4.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_56000_58000_B1_Site-347.ch5.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_56000_58000_B1_Site-347.ch6.tif
58004
['B1_site348.']
58056
['B1_site349.']
58320
['B1_site350.']
58387
['B1_site351.']
58701
['B1_site352.']
58864
['B1_site353.']
59128
['B1_site354.']
59154
['B1_site355.']
59339
['B1_site356.']
59429
['B1_site357.']
59642
['B1_site358.']
596

74551
['B1_site460.']
74646
['B1_site461.']
74877
['B1_site462.']
74968
['B1_site463.']
75254
['B1_site464.']
75371
['B1_site465.']
75586
['B1_site466.']
75725
['B1_site467.']
75982
['B1_site468.']
saving
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_74000_76000_B1_Site-468.ch0.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_74000_76000_B1_Site-468.ch1.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_74000_76000_B1_Site-468.ch2.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_74000_76000_B1_Site-468.ch3.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_74000_76000_B1_Site-468.ch4.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_74000_76000_B1_Site-468.ch5.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_74000_76000_B1_Site-468.ch6.tif
76117
['B1_site469.']
76269
['B1_site470.']
76344
['B1_site471.']
76496
['B1_site472.']
76529
['B1_site473.']
76682
['B1_site474.']
76757
['B1_site475.']
76964
['B1_site476.']
770

...

129468
['B1_site799.']
129616
['B1_site800.']
129691
['B1_site801.']
129880
['B1_site802.']
saving
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_128000_130000_B1_Site-802.ch0.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_128000_130000_B1_Site-802.ch1.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_128000_130000_B1_Site-802.ch2.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_128000_130000_B1_Site-802.ch3.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_128000_130000_B1_Site-802.ch4.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_128000_130000_B1_Site-802.ch5.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_128000_130000_B1_Site-802.ch6.tif
130009
['B1_site803.']
130319
['B1_site804.']
130487
['B1_site805.']
130770
['B1_site806.']
130892
['B1_site807.']
131130
['B1_site808.']
131232
['B1_site809.']
131427
['B1_site810.']
131551
['B1_site811.']
131673
['B1_site812.']
131748
['B1_site813.']
saving
/home/rcarl

146572
['B1_site906.']
146622
['B1_site907.']
146887
['B1_site908.']
146936
['B1_site909.']
147164
['B1_site910.']
147220
['B1_site911.']
147450
['B1_site912.']
147528
['B1_site913.']
147737
['B1_site914.']
147772
['B1_site915.']
147946
['B1_site916.']
saving
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_146000_148000_B1_Site-916.ch0.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_146000_148000_B1_Site-916.ch1.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_146000_148000_B1_Site-916.ch2.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_146000_148000_B1_Site-916.ch3.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_146000_148000_B1_Site-916.ch4.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_146000_148000_B1_Site-916.ch5.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_146000_148000_B1_Site-916.ch6.tif
148046
['B1_site917.']
148231
['B1_site918.']
148289
['B1_site919.']
148454
['B1_site920.']
148529
['B1_site92

/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_162000_164000_B1_Site-1030.ch2.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_162000_164000_B1_Site-1030.ch3.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_162000_164000_B1_Site-1030.ch4.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_162000_164000_B1_Site-1030.ch5.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_162000_164000_B1_Site-1030.ch6.tif
164023
['B1_site1031.']
164278
['B1_site1032.']
164443
['B1_site1033.']
164693
['B1_site1034.']
164905
['B1_site1035.']
165159
['B1_site1036.']
165286
['B1_site1037.']
165490
['B1_site1038.']
165559
['B1_site1039.']
165785
['B1_site1040.']
165879
['B1_site1041.']
saving
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_164000_166000_B1_Site-1041.ch0.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_164000_166000_B1_Site-1041.ch1.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_164000_166000_B1_Site-1041.ch2.tif
/

180698
['B1_site1139.']
180743
['B1_site1140.']
180920
['B1_site1141.']
180920
['B1_site1142.']
181168
['B1_site1143.']
181264
['B1_site1144.']
181489
['B1_site1145.']
181578
['B1_site1146.']
181746
['B1_site1147.']
181831
['B1_site1148.']
saving
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_180000_182000_B1_Site-1148.ch0.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_180000_182000_B1_Site-1148.ch1.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_180000_182000_B1_Site-1148.ch2.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_180000_182000_B1_Site-1148.ch3.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_180000_182000_B1_Site-1148.ch4.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_180000_182000_B1_Site-1148.ch5.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_180000_182000_B1_Site-1148.ch6.tif
182056
['B1_site1149.']
182149
['B1_site1150.']
182417
['B1_site1151.']
182526
['B1_site1152.']
182829
['B1_site1153

/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_196000_198000_B1_Site-1259.ch2.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_196000_198000_B1_Site-1259.ch3.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_196000_198000_B1_Site-1259.ch4.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_196000_198000_B1_Site-1259.ch5.tif
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_196000_198000_B1_Site-1259.ch6.tif
198010
['B1_site1260.']
198224
['B1_site1261.']
198224
['B1_site1262.']
198224
['B1_site1263.']
198407
['B1_site1264.']
198407
['B1_site1265.']
198477
['B1_site1266.']
198477
['B1_site1267.']
198622
['B1_site1268.']
198622
['B1_site1269.']
198713
['B1_site1270.']
198746
['B1_site1271.']
198915
['B1_site1272.']
198966
['B1_site1273.']
199083
['B1_site1274.']
199223
['B1_site1275.']
199446
['B1_site1276.']
199528
['B1_site1277.']
199622
['B1_site1278.']
199622
['B1_site1279.']
199876
['B1_site1280.']
saving last site and cell
(1968, 7, 299, 

In [47]:
plate = "M40A"
well = "A3"
site = 1256
cell = 935
ix = df_26.query('well == @well & plate == @plate & site == @site & cell == @cell').index[0]
ofint = df_26.copy().iloc[ix+1:,:].query('well == @well & plate == @plate')
save_singlecells_finish(
                 plate = plate, well = well, start = 144000,
                 nchannels = 7, 
                df_cells_interest = ofint)


####

# sites
1281
144000
['A3_site0.']
144000
['A3_site1.']
144000
['A3_site2.']
144000
['A3_site3.']
144000
['A3_site4.']
144000
['A3_site5.']
144000
['A3_site6.']
144000
['A3_site7.']
144000
['A3_site8.']
144000
['A3_site9.']
144000
['A3_site10.']
144000
['A3_site11.']
144000
['A3_site12.']
144000
['A3_site13.']
144000
['A3_site14.']
144000
['A3_site15.']
144000
['A3_site16.']
144000
['A3_site17.']
144000
['A3_site18.']
144000
['A3_site19.']
144000
['A3_site20.']
144000
['A3_site21.']
144000
['A3_site22.']
144000
['A3_site23.']
144000
['A3_site24.']
144000
['A3_site25.']
144000
['A3_site26.']
144000
['A3_site27.']
144000
['A3_site28.']
144000
['A3_site29.']
144000
['A3_site30.']
144000
['A3_site31.']
144000
['A3_site32.']
144000
['A3_site33.']
144000
['A3_site34.']
144000
['A3_site35.']
144000
['A3_site36.']
144000
['A3_site37.']
144000
['A3_site38.']
144000
['A3_site39.']
144000
['A3_site40.']
144000
['A3_site41.']
144000
['A3_site42.']
144000
['A3_site43.']
144000
['A3_site44.']
144000


144000
['A3_site434.']
144000
['A3_site435.']
144000
['A3_site436.']
144000
['A3_site437.']
144000
['A3_site438.']
144000
['A3_site439.']
144000
['A3_site440.']
144000
['A3_site441.']
144000
['A3_site442.']
144000
['A3_site443.']
144000
['A3_site444.']
144000
['A3_site445.']
144000
['A3_site446.']
144000
['A3_site447.']
144000
['A3_site448.']
144000
['A3_site449.']
144000
['A3_site450.']
144000
['A3_site451.']
144000
['A3_site452.']
144000
['A3_site453.']
144000
['A3_site454.']
144000
['A3_site455.']
144000
['A3_site456.']
144000
['A3_site457.']
144000
['A3_site458.']
144000
['A3_site459.']
144000
['A3_site460.']
144000
['A3_site461.']
144000
['A3_site462.']
144000
['A3_site463.']
144000
['A3_site464.']
144000
['A3_site465.']
144000
['A3_site466.']
144000
['A3_site467.']
144000
['A3_site468.']
144000
['A3_site469.']
144000
['A3_site470.']
144000
['A3_site471.']
144000
['A3_site472.']
144000
['A3_site473.']
144000
['A3_site474.']
144000
['A3_site475.']
144000
['A3_site476.']
144000
['A3

144000
['A3_site840.']
144000
['A3_site841.']
144000
['A3_site842.']
144000
['A3_site843.']
144000
['A3_site844.']
144000
['A3_site845.']
144000
['A3_site846.']
144000
['A3_site847.']
144000
['A3_site848.']
144000
['A3_site849.']
144000
['A3_site850.']
144000
['A3_site851.']
144000
['A3_site852.']
144000
['A3_site853.']
144000
['A3_site854.']
144000
['A3_site855.']
144000
['A3_site856.']
144000
['A3_site857.']
144000
['A3_site858.']
144000
['A3_site859.']
144000
['A3_site860.']
144000
['A3_site861.']
144000
['A3_site862.']
144000
['A3_site863.']
144000
['A3_site864.']
144000
['A3_site865.']
144000
['A3_site866.']
144000
['A3_site867.']
144000
['A3_site868.']
144000
['A3_site869.']
144000
['A3_site870.']
144000
['A3_site871.']
144000
['A3_site872.']
144000
['A3_site873.']
144000
['A3_site874.']
144000
['A3_site875.']
144000
['A3_site876.']
144000
['A3_site877.']
144000
['A3_site878.']
144000
['A3_site879.']
144000
['A3_site880.']
144000
['A3_site881.']
144000
['A3_site882.']
144000
['A3

144000
['A3_site1239.']
144000
['A3_site1240.']
144000
['A3_site1241.']
144000
['A3_site1242.']
144000
['A3_site1243.']
144000
['A3_site1244.']
144000
['A3_site1245.']
144000
['A3_site1246.']
144000
['A3_site1247.']
144000
['A3_site1248.']
144000
['A3_site1249.']
144000
['A3_site1250.']
144000
['A3_site1251.']
144000
['A3_site1252.']
144000
['A3_site1253.']
144000
['A3_site1254.']
144000
['A3_site1255.']
144000
['A3_site1256.']
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
(43, 14)
144041
['A3_site1257.']
(151, 14)
(151, 14)
(151, 14)
(151, 14)
(151, 14)
(151, 14)
(151, 14)
(151, 14)
(151, 14)
(151, 14)
(151, 14)
(151, 14)
(151, 14)
(151, 14)
(151, 14)
(151, 14)
(151, 14)
(151,

(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)
(372, 14)


(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
(54, 14)
145635
['A3_site1278.']
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
(24, 14)
145659
['A3_site1279.']
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(164, 14)
(

In [10]:
import gc

# takes about 7min/ch

files =  natsorted(glob('/home/rcarlson/mountb/m40-m42/process/single_cells/*M40A*166000_168000*B2*.tif'))[-1:]
for file in files:
    channel = file.split('ch')[1][0]
    print('ch: ', channel)
    plate = file.split('/')[-1].split('_')[0]
    well = file.split('/')[-1].split('_')[3]
    start = file.split('/')[-1].split('_')[1]
    end = file.split('/')[-1].split('_')[2]

    print(start, end, plate, well)
    batch_preprocess_predict(file, start=start,end=end, plate=plate, well=well) ## 
    gc.collect()
    
    
    ## 38 - 1-2 left.



ch:  6
166000 168000 M40A B2
/home/rcarlson/mountb/m40-m42/process/single_cells/M40A_166000_168000_B2_Site-1184.ch6.tif
(2000, 299, 299)
mean, sd
5533.593 2533.048
0
500
1000
1500
preprocessing done
2000
tf dataset created
2000/2000 [==============================] - 101s 48ms/step
